In [10]:
import argparse
import os
from os import listdir
from os.path import isfile, join

import numpy as np
import math

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from skimage import io, transform
import matplotlib.pyplot as plt


In [22]:
torch.cuda.is_available()
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

labels = {}
normal_ids = listdir('./drive/My Drive/chest_xray/train/NORMAL')
labels = dict(zip(normal_ids, np.zeros(len(normal_ids), dtype='int')))
pneumonia_ids = listdir('./drive/My Drive/chest_xray/train/PNEUMONIA')
labels.update(dict(zip(pneumonia_ids, np.ones(len(pneumonia_ids), dtype='int'))))

In [26]:
data_path = './drive/My Drive/chest_xray/train'
train_dataset = torchvision.datasets.ImageFolder(root=data_path, 
                                transform=transforms.Compose([
                               transforms.Resize(IMAGE_SIZE),
                               #transforms.CenterCrop(IMAGE_SIZE),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                         shuffle=True)
train_iter = iter(train_dataloader)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img
